In [1]:
from subprocess import run, Popen
from pathlib import Path
import pandas as pd
import json, os
from inspect import cleandoc

agencies = pd.read_csv(Path("agencies.csv").open()).agency.unique()
agencymapping = pd.read_csv(Path("mappings.csv").open())

In [ ]:
report_notebooks = Path(".").glob("report-*.ipynb")
processes = []
for report in report_notebooks:
    for agency in agencies:
        agency_data = agencymapping[agencymapping.Agency_Short.str.contains(agency, case=False)]
        agency_name = agency_data.SearchKey.max()
        workdir = Path("reports") / agency # Standard dir for notebook to output files into
        workdir.mkdir(exist_ok=True)
        nbfile = workdir / report.name.replace("report-","")
        nbfile.write_text(report.read_text())
        report_env = os.environ.copy()
        report_env["agency"] = agency
        report_env["agency_name"] = agency_name
        report_env["report_date"] = "29 March 2022"
        processes.append(Popen(["jupyter", "nbconvert", nbfile.name, "--to", "html", "--execute", "--no-prompt", "--no-input"], cwd=workdir, env=report_env))
        if len(processes) >= 4: # run 4 exports in parallel
            exitcodes = [p.wait() for p in processes]
            processes = []

exitcodes = [p.wait() for p in processes]